In [1]:
# compile, must use -fPIC

In [2]:
!gfortran -O0 -g -c testmod.f90 -o testmod.o -fcheck=all -fmax-errors=1 -fPIC -cpp

In [3]:
# link, must use -shared to create a .so (equivalent to the windows .dll)

In [4]:
!gfortran -shared testmod.o -o testmod.so

In [5]:
from ctypes import * #cdll, CDLL, Array
lib = cdll.LoadLibrary('./testmod.so')

In [6]:
print(lib)

<CDLL './testmod.so', handle 561134d64240 at 0x7f777c21b630>


# globals

In [7]:
#https://docs.python.org/3/library/ctypes.html#accessing-values-exported-from-dlls
print(type(lib.a))
print(lib.a)
# lib.a() #The kernel appears to have died. It will restart automatically.

py_a = c_double.in_dll(lib, "a")
print(type(py_a))
print(py_a)
print(py_a.value)

<class 'ctypes.CDLL.__init__.<locals>._FuncPtr'>
<class 'ctypes.c_double'>
c_double(653.0)
653.0


In [8]:
#https://docs.python.org/3/library/ctypes.html#accessing-values-exported-from-dlls
print(lib.b)
# lib.b() #The kernel appears to have died. It will restart automatically.

py_b = pointer((c_double*3).in_dll(lib, "b"))
print(type(py_b))
print(py_b)
print(py_b.contents)
print(py_b.contents[0])
print(py_b.contents[1])
print(py_b.contents[2])

<class '__main__.LP_c_double_Array_3'>
1.0
2.0
3.0


# subroutines

## void sub(void) with global effect    

In [9]:
#BEFORE
print(py_b.contents[0])
print(py_b.contents[1])
print(py_b.contents[2])
print(py_a.value)

#THEN
#call
print(lib.testhw)
void=lib.testhw()

print(py_b.contents[0])
print(py_b.contents[1])
print(py_b.contents[2])
print(py_a.value)


1.0
2.0
3.0
653.0
2.0
3.0
4.0
654.0


# load pretty constants

In [10]:
from math import sin, pi
print(sin(pi))

1.2246467991473532e-16


# subroutine receiving value

In [11]:
#BEFORE
print(py_b.contents[0])
print(py_b.contents[1])
print(py_b.contents[2])
print(py_a.value)

#THEN
x1 = c_double (pi)
x2 = c_double (pi*2)
x3 = c_double (pi*3)
x4 = c_double (pi*4)

#safety
lib.testsubval.argtypes=[c_double, c_double,c_double,c_double]
#badcall
try:
    void=lib.testsubval ( )
except Exception as e:
    print(e)
#goodcall
void=lib.testsubval(x1,x2,x3,x4)

print(py_a.value)
print(py_b.contents[0])
print(py_b.contents[1])
print(py_b.contents[2])

2.0
3.0
4.0
654.0
this function takes at least 4 arguments (0 given)
3.141592653589793
6.283185307179586
9.42477796076938
12.566370614359172


# subroutine receiving reference

## ctypes exports the byref() function which is used to pass parameters by reference
> x and y are scalars

In [12]:
x = c_double (pi)
y = c_double (1)
#safety
lib.testsub.argtypes=[(c_double*1), (c_double*1)]
#badcall
try:
    void=lib.testsub ( (x),  (y))
except Exception as e:
    print(e)
#goodcall
void=lib.testsub (byref (x), byref (y))
#results
print(type(void))
print(void) #what does this integer mean?
print(x.value)
print(y.value)

argument 1: <class 'TypeError'>: expected c_double_Array_1 instance instead of c_double
<class 'int'>
2082678928
3.141592653589793
1.2246467991473532e-16


# with array
> x and y are 1 element arrays

In [13]:
x = (c_double*1) (pi)
y = (c_double*1) (1)
#safety
lib.testsub.argtypes=[(c_double*1), (c_double*1)]
#badcall
try:
    void=lib.testsub ( byref(x),  byref(y))
except Exception as e:
    print(e)
#goodcall
void=lib.testsub ( (x),  (y))
#results
print(type(void))
print(void) #what does this integer mean?
print(x[0])
print(y[0])

argument 1: <class 'TypeError'>: expected c_double_Array_1 instance instead of pointer to c_double_Array_1
<class 'int'>
2082677976
3.141592653589793
1.2246467991473532e-16


## The same effect can be achieved with the pointer() function, although pointer() does a lot more work since it constructs a real pointer object
> x and y are 1 pointers

In [14]:
x = c_double (pi)
y = c_double (1)
#safety
#ctypes.POINTER(type)
#This factory function creates and returns a new ctypes pointer type. Pointer types are cached and reused internally, so calling this function repeatedly is cheap. type must be a ctypes type.
lib.testsub.argtypes=[POINTER(c_double), POINTER(c_double)]
#badcall
# try:
#     void=lib.testsub ( (x),  (y))
# except Exception as e:
#     print(e)
#goodcall
#This function creates a new pointer instance, pointing to obj. The returned object is of the type POINTER(type(obj)).
#  Note: If you just want to pass a pointer to an object to a foreign function call, you should use byref(obj) which is much faster.
px=pointer(x)
py=pointer(y)
void=lib.testsub (px, py)
#results
print(type(void))
print(void) #what does this integer mean?
print(x.value)
print(y.value)

<class 'int'>
2082680016
3.141592653589793
1.2246467991473532e-16


## subroutine processing one lengthy array

In [15]:
res=(c_double*10)(-1,-2, -3, -4, -5, -6, -7, -8, -9, -10)
print(type(res))
print(res)
print(res[0])
print(res[1])
print(res[2])

lib.testsub2.argtypes=[c_double*10]

#badcall
try:
    void=lib.testsub2(byref (res))
except Exception as e:
    print(e)
#goodcall
    void=lib.testsub2((res))

print(type(res))
print(res)
print(res[0])
print(res[1])
print(res[2])

<class '__main__.c_double_Array_10'>
-1.0
-2.0
-3.0
argument 1: <class 'TypeError'>: expected c_double_Array_10 instance instead of pointer to c_double_Array_10
<class '__main__.c_double_Array_10'>
1.0
2.0
3.0


## subroutine processing two lengthy array

In [16]:
a1 = (c_double * 10)(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
a2 = (c_double * 10)(-1,-2, -3, -4, -5, -6, -7, -8, -9, -10)

print(type(a1))
print(a1)
print(a1[0])
print(a1[1])
print(a1[2])


print(type(a2))
print(a2)
print(a2[0])
print(a2[1])
print(a2[2])


lib.testsub3.argtypes=[c_double*10,c_double*10]

#badcall
try:
    void=lib.testsub3(byref (a1), byref (a2))
except Exception as e:
    print(e)
#goodcall
    void=lib.testsub3(a1,a2)

print(type(a1))
print(a1)
print(a1[0])
print(a1[1])
print(a1[2])


print(type(a2))
print(a2)
print(a2[0])
print(a2[1])
print(a2[2])


<class '__main__.c_double_Array_10'>
1.0
2.0
3.0
<class '__main__.c_double_Array_10'>
-1.0
-2.0
-3.0
argument 1: <class 'TypeError'>: expected c_double_Array_10 instance instead of pointer to c_double_Array_10
<class '__main__.c_double_Array_10'>
1.0
2.0
3.0
<class '__main__.c_double_Array_10'>
2.0
4.0
6.0


## via numpy

In [17]:
#https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.ctypeslib.html
import numpy as np
import numpy.ctypeslib as nct

### An ndpointer instance is used to describe an ndarray in restypes and argtypes specifications. This approach is more flexible than using, for example, POINTER(c_double), since several restrictions can be specified, which are verified upon calling the ctypes function. These include data type, number of dimensions, shape and flags. If a given array does not satisfy the specified restrictions, a TypeError is raised.
(https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.ctypeslib.html)

In [18]:
npa1=np.array([1,2,3],dtype=np.float64)
npa2=np.array([0,0,0],dtype=np.float64)

print(type(npa1))
print(type(npa2))

print(npa1)
print(npa2)

lib.testsub3.argtypes = [np.ctypeslib.ndpointer(dtype=np.float64,
                                                  ndim=1,
                                                  flags='C_CONTIGUOUS'),
                                 np.ctypeslib.ndpointer(dtype=np.float64,
                                                  ndim=1,
                                                  flags='C_CONTIGUOUS')]
 

void=lib.testsub3(npa1,npa2)

print(type(npa1))
print(type(npa2))

print(npa1)
print(npa2)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[ 1.  2.  3.]
[ 0.  0.  0.]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[ 1.  2.  3.]
[ 2.  4.  6.]


In [19]:
print(type(npa1))
print(type(npa2))

print(npa1)
print(npa2)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[ 1.  2.  3.]
[ 2.  4.  6.]


# function

In [ ]:
argin = c_double(1)
print(argin.value)

res = c_double(1)
print(res.value)


1.0
1.0


In [ ]:
libm = cdll.LoadLibrary(util.find_library("m"))

In [ ]:
libm.sin.restype = c_double
libm.sin.argtypes = [c_double]
ans=libm.sin(argin)
print(ans)

In [ ]:
lib.sin.restype = c_double
lib.sin.argtypes = [c_double]
ans=lib.sin(argin)
print(ans)

In [ ]:
lib.testfunctionbyval.restype = c_double
lib.testfunctionbyval.argtypes = [c_double]
ans=lib.testfunctionbyval(argin)
print(ans)

In [ ]:
lib.testfunctionbyref.restype = c_double
lib.testfunctionbyref.argtypes = [c_double*1]
ans=lib.testfunctionbyref(byref(argin))
print(ans)

In [ ]:
lib.testsizearrayn.argtypes = [c_int, POINTER(c_int*10), (c_int*1)]

v=(c_int*10)(-1,-2, -3, -4, -5, -6, -7, -8, -9, -10)
pv=pointer(v)

ci = c_int(1213)

void=lib.testsizearrayn(10, pv, byref(ci))
print(ci.value)

In [ ]:
#static argtypes
lib.testsizearray2.argtypes = [ (c_int*10),
                              (c_int*1)]

v=(c_int*10)(-1,-2, -3, -4, -5, -6, -7, -8, -9, -10)


ci = c_int(1213)

void=lib.testsizearray2( v, byref(ci))
print(ci.value)

In [ ]:
stop

In [ ]:
#dynamic argtypes

lib.testsizearray3.argtypes = [ np.ctypeslib.ndpointer(dtype=np.int,
                                                  ndim=1,
                                                  flags='C_CONTIGUOUS'),
                              (c_int*1)]

v=np.array([-1,-2, -3, -4, -5, -6, -7, -8, -9, -10],dtype=np.int)


ci = c_int(1213)

void=lib.testsizearray3( v, byref(ci))
print(ci.value)

In [ ]:
stop

In [ ]:
#dynamic argtypes
# npa1=np.array([1,2,3],dtype=np.float64)
# lib.testsub3.argtypes = [np.ctypeslib.ndpointer(dtype=np.float64,
#                                                   ndim=1,
#                                                   flags='C_CONTIGUOUS'),


lib.testsizearray3b.argtypes = [ c_int,
                                
                                np.ctypeslib.ndpointer(dtype=np.float64,
                                                  ndim=1,
                                                  flags='C_CONTIGUOUS'),
                              (c_int*1)]

v=np.array([-1,-2, -3, -4, -5, -6, -7, -8, -9, -10],dtype=np.float64)


ci = c_int(1213)

void=lib.testsizearray3b( 10,v, byref(ci))
print(ci.value)

#ESSA 3B TAVA DANDO ERRADO COM O INT, VER COMPATIBILIDADE DE TIPOS ENTRE NP.INT E C_INT

In [ ]:
lib.testsizearray1.argtypes = [ (c_int*10),
                              (c_int*1)]

v=(c_int*10)(-1,-2, -3, -4, -5, -6, -7, -8, -9, -10)


ci = c_int(1213)

void=lib.testsizearray1( v, byref(ci))
print(ci.value)

In [ ]:
# char e string

lib.testhelloworld.argtypes = [ c_int, (c_char_p), (c_char)*1]

mychar= c_char(b"A")
mystring=c_char_p(b"HELLOWORLD")
N=c_int(len(mystring.value))
print(N.value)
void=lib.testhelloworld( N, (mystring), byref(mychar))




In [ ]:
# char e string

lib.testanswer.argtypes = [ c_int, (c_char_p), (c_char)*1]

mychar= c_char(b"A")
mystring=c_char_p(b"HELLOWORLD")
N=c_int(len(mystring.value))
print(N.value)
void=lib.testanswer( N, (mystring), byref(mychar))




In [ ]:
print(mystring)
print(mystring.value)
print(mychar)
print(mychar.value)

# test allocatable

In [ ]:

lib.testallocatable.argtypes = [ (c_int)*1, POINTER(c_int)] #times any number?

myint= c_int(0)

print(myint.value)

mypointer= POINTER(c_int)() #null_ptr

# void=lib.testallocatable( byref(myint), mypointer)




In [ ]:
#i guess this ones need c_ptr and c_loc
void=lib.testallocatable( byref(myint), byref(mypointer))


In [ ]:
!pcmanfm